<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Interacting with Azure SQL Servers - Part 1

## Azure Actions Covered

* Connecting to an Azure SQL Server with pyodbc
* Reading and retrieving data from a SQL database
* Inserting data to a SQL database

In this lecture, we'll learn how to interact with Azure SQL Servers with Python.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. AZURE_SUBSCRIPTION_ID).

In [1]:
from azure.identity import AzureCliCredential
from azure.mgmt.sql import SqlManagementClient

import pyodbc

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

We'll instantiate our credential as well as our `SqlManagementClient`.

In [2]:
credential = AzureCliCredential()
sql_client = SqlManagementClient(credential, AZURE_SUBSCRIPTION_ID)

Let's set up the connection string again for our SQL Server.

In [3]:
SERVER = 'bens-server.database.windows.net'
DATABASE = 'bens-db'
USERNAME = 'benadmin'
PASSWORD = 'testpassword123!'
# Has to be installed
DRIVER = '{ODBC Driver 18 for SQL Server}'

connection_string = f'DRIVER={DRIVER};PORT=1433;SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

Let's try our connection string to see if we can connect.

In [ ]:
# connection = pyodbc.connect(connection_string)
# cursor = connection.cursor()

It could be that we need to create a new firewall rule to connect to our DB, so let's use the IP address from the error message.

In [5]:
firewall = sql_client.firewall_rules.create_or_update(
    DEFAULT_RESOURCE_GROUP,
    'bens-server',
    'JupyterNotebook2',
    {
        'start_ip_address': '72.174.17.20',
        'end_ip_address': '72.174.17.20'
    }
)


Now we can retry the connection.

In [17]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

## Reading and Retrieving Data

Reading and retrieving data from a SQL Server follows a common Python pattern. First, we execute a SQL statement. Then, we can call a "fetch" method on our cursor to retrieve the data and interact with it.

Let's select the first 10 items in our `Address` table and use a `while` loop to show all the retrieved data.

In [8]:
cursor.execute(
"""
select top (10) *
from [SalesLT].[Address]
"""
)
row = cursor.fetchone()
while row:
    print(str(row))
    row = cursor.fetchone()

(9, '8713 Yosemite Ct.', None, 'Bothell', 'Washington', 'United States', '98011', '268AF621-76D7-4C78-9441-144FD139821A', datetime.datetime(2006, 7, 1, 0, 0))
(11, '1318 Lasalle Street', None, 'Bothell', 'Washington', 'United States', '98011', '981B3303-ACA2-49C7-9A96-FB670785B269', datetime.datetime(2007, 4, 1, 0, 0))
(25, '9178 Jumping St.', None, 'Dallas', 'Texas', 'United States', '75201', 'C8DF3BD9-48F0-4654-A8DD-14A67A84D3C6', datetime.datetime(2006, 9, 1, 0, 0))
(28, '9228 Via Del Sol', None, 'Phoenix', 'Arizona', 'United States', '85004', '12AE5EE1-FC3E-468B-9B92-3B970B169774', datetime.datetime(2005, 9, 1, 0, 0))
(32, '26910 Indela Road', None, 'Montreal', 'Quebec', 'Canada', 'H1Y 2H5', '84A95F62-3AE8-4E7E-BBD5-5A6F00CD982D', datetime.datetime(2006, 8, 1, 0, 0))
(185, '2681 Eagle Peak', None, 'Bellevue', 'Washington', 'United States', '98004', '7BCCF442-2268-46CC-8472-14C44C14E98C', datetime.datetime(2006, 9, 1, 0, 0))
(297, '7943 Walnut Ave', None, 'Renton', 'Washington', 'Un

We can take a look at the tables in our schema using `cursor.tables()`.

In [9]:
tables = cursor.tables(schema='SalesLT', tableType='TABLE')
for table in tables:
    print(table.table_name)
print(cursor.messages)
print(cursor.rowcount)

Address
Customer
CustomerAddress
Product
ProductCategory
ProductDescription
ProductModel
ProductModelProductDescription
SalesOrderDetail
SalesOrderHeader
[]
-1


We can also use `cursor.statistics()` to show the column names on a particular table.

In [13]:
table_stats = cursor.statistics(table='Address', schema='SalesLT')
for row in table_stats:
    print(f'{row.column_name}')

None
AddressID
rowguid
AddressLine1
AddressLine2
City
StateProvince
PostalCode
CountryRegion
StateProvince


Let's update our previous query to take advantage of variables - which protect us against SQL injection - as well as the dictionary syntax to reference returned column names.

In [18]:
result = cursor.execute(
"""
select top (?) *
from [SalesLT].[Address]
""", 10
)
for row in result:
    print(f'{row.AddressLine1}, {row.City}, {row.StateProvince} {row.PostalCode} {row.CountryRegion}')

8713 Yosemite Ct., Bothell, Washington 98011 United States
1318 Lasalle Street, Bothell, Washington 98011 United States
9178 Jumping St., Dallas, Texas 75201 United States
9228 Via Del Sol, Phoenix, Arizona 85004 United States
26910 Indela Road, Montreal, Quebec H1Y 2H5 Canada
2681 Eagle Peak, Bellevue, Washington 98004 United States
7943 Walnut Ave, Renton, Washington 98055 United States
6388 Lake City Way, Burnaby, British Columbia V5A 3A6 Canada
52560 Free Street, Toronto, Ontario M4B 1V7 Canada
22580 Free Street, Toronto, Ontario M4B 1V7 Canada


## Inserting Data

To insert data into our SQL database, we need to first execute the insert statement and pass the relevant data. Then, we'll commit the statement to the database.

In [21]:
insert_stmt = cursor.execute("""
insert into SalesLT.Address 
(AddressLine1, AddressLine2, City, StateProvince, CountryRegion, PostalCode) 
values (?,?,?,?,?,?)
""",
'5 Easy Street', '', 'Austin', 'Tennessee', 'United States', '55667')
connection.commit()

We can use the `rowcount` attribute to verify the number of rows inserted into the database.

In [22]:
insert_stmt.rowcount

1

Let's retrieve that new data from our database as well.

In [26]:
result = cursor.execute(
"""
select *
from [SalesLT].[Address]
where [SalesLT].[Address].[AddressLine1] = ?
""", '5 Easy Street'
)

In [29]:
result.fetchall()

[(11383, '5 Easy Street', '', 'Austin', 'Tennessee', 'United States', '55667', '2670B2E6-DE5D-418D-8417-453F95547865', datetime.datetime(2023, 4, 26, 16, 35, 26, 670000)),
 (11384, '5 Easy Street', '', 'Austin', 'Tennessee', 'United States', '55667', '8F3B6503-F882-46F0-B649-06B5AB07B44C', datetime.datetime(2023, 4, 26, 16, 35, 49, 863000)),
 (11385, '5 Easy Street', '', 'Austin', 'Tennessee', 'United States', '55667', 'B5C3F2A8-F7F0-49C9-9F78-80966EDFFF64', datetime.datetime(2023, 4, 26, 16, 36, 10))]

In the next lecture, we'll continue learning about the ways to interact with data in an Azure SQL Server via Python.